In [3]:
# Sample Problem 1 (Ex 1 from Dynopt Guide)
#
#       min X2(tf)
#       s.t.    X1_dot = u                      X1(0) = 1
#               X2_dot = X1^2 + u^2             X2(0) = 0
#               tf = 1
from pyomo.environ import *
from pyomo.dae import *
m = ConcreteModel()
m.t = ContinuousSet(bounds=(0,1))
m.x1 = Var(m.t, bounds=(0,1))
m.x2 = Var(m.t, bounds=(0,1))
m.u = Var(m.t, initialize=0)
m.x1dot = DerivativeVar(m.x1)
m.x2dot = DerivativeVar(m.x2)
m.obj = Objective(expr=m.x2[1])
def _x1dot(M,i):
        if i == 0:
                return Constraint.Skip
        return M.x1dot[i] == M.u[i]
m.x1dotcon = Constraint(m.t, rule=_x1dot)
def _x2dot(M,i):
        if i == 0:
                return Constraint.Skip
        return M.x2dot[i] == M.x1[i]**2 + M.u[i]**2
m.x2dotcon = Constraint(m.t, rule=_x2dot)
def _init(M):
        yield M.x1[0] == 1
        yield M.x2[0] == 0
        yield ConstraintList.End
m.init_conditions = ConstraintList(rule=_init)

In [7]:
from pyomo.environ import *
from pyomo.dae import *
from pyomo.opt import SolverFactory
from pyomo.dae.plugins.finitedifference import Finite_Difference_Transformation
from pyomo.dae.plugins.colloc import Collocation_Discretization_Transformation

instance = m
# Discretize model using Backward Finite Difference method
#discretize = Finite_Difference_Transformation()
#disc_instance = discretize.apply(instance,nfe=20,scheme='BACKWARD')
# Discretize model using Orthogonal Collocation
discretize = Collocation_Discretization_Transformation()
disc_instance = discretize.apply(instance,nfe=10,ncp=5,scheme='LAGRANGE-LEGENDRE')
disc_instance = discretize.reduce_collocation_points(var=instance.u,
        ncp=2, diffset=instance.t)
solver='ipopt'
opt=SolverFactory(solver)
results = opt.solve(disc_instance,tee=True)
disc_instance.load(results)
x1 = []
x2 = []
u = []
t=[]

for i in sorted(disc_instance.t):
    t.append(i)
    x1.append(value(disc_instance.x1[i]))
    x2.append(value(disc_instance.x2[i]))
    u.append(value(disc_instance.u[i]))
import matplotlib.pyplot as plt
plt.plot(t,x1)
plt.plot(t,x2)
plt.show()
plt.plot(t,u)
plt.show()


AttributeError: 'Collocation_Discretization_Transformation' object has no attribute 'reduce_collocation_points'